<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=43dc24c99a2f3522065aca689eae0fa0b9d0d5f9887a0469c7addc9d643f3231
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-17 16:08:53
-------------------
qualified stocks: 88
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.60 K
Current:  1.46 C
-------------------
Today PnL: 51.39 K (0.35%)
Current PnL: -20.07 L (-13.15%)
CY Booked + Current PnL: -6.11 L (-4.0%)
-------------------
Total profit:  1.76 L
Total loss:  -21.83 L
-------------------
Total Booked + Current PnL: 20.87 L (16.61%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.58%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.37 C
Est FTT PnL: 90.90 L (62.36%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.61%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,150.00,85.0,M-SC,0.64,163841.0,-10718.0,10748.0,1.73,-6.14,6.56,0.02,234.0,-1.00,1.12,42.51,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,101.92,52.0,M-SC,3.43,87030.0,-13747.0,13838.0,0.12,-13.64,15.90,0.09,253.0,-0.99,0.60,14.34,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-15.14,48.0,X-MC,3.56,88680.0,-1998.0,16902.0,0.75,-2.20,19.06,16.44,63.0,-0.12,0.61,17.53,X40,ATH,PHARMA
35,ICICIGI,2252.93,-14.60,59.0,X-MC,7.54,189990.0,15670.0,19531.0,1.89,8.99,10.28,20.19,68.0,0.80,1.30,25.46,X40,ATH,INSURANCE
43,ITC,452.00,-40.74,47.0,X-LC,0.91,199072.0,-1066.0,21958.0,-0.26,-0.53,11.03,10.44,5.0,-0.05,1.37,5.00,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.0,93.51,52.0,H-SC,16.79,131123.0,-87373.0,115559.0,9.01,-39.99,88.13,12.90,161.0,-0.76,0.90,9.03,OX40N,NTT,FINANCE
71,SURYODAY,240.0,60.09,51.0,H-SC,13.53,147390.0,-31681.0,97410.0,5.07,-17.69,66.09,36.71,167.0,-0.33,1.01,46.24,XR,NTT,BANKS
67,SIEMENS,4671.5,-0.76,63.0,H-LC,1.54,161630.0,-24465.0,71942.0,4.81,-13.15,44.51,25.51,49.0,-0.34,1.11,19.11,AR,ATH,ELECTRICAL
63,SAMMAANCAP,326.0,22.90,61.0,M-SC,37.08,165294.0,16074.0,128103.0,4.27,10.77,77.50,96.62,245.0,0.13,1.13,78.88,XY25,NTT,FINANCE
56,QUESS,986.0,-52.07,35.0,X-SC,37.19,47663.0,-17343.0,168269.0,2.74,-26.68,353.04,232.18,83.0,-0.10,0.33,2.74,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.0,-51.71,31.0,L-SC,21.15,70234.0,-43315.0,83368.0,-4.74,-38.15,118.70,35.27,270.0,-0.52,0.48,71.90,XR,NTT,HOTELS
77,TMPV,1065.0,-20.63,25.0,X-LC,0.27,244118.0,-32292.0,453449.0,-4.73,-11.68,185.75,152.37,3.0,-0.07,1.67,0.00,XY24,NTT,AUTO
65,SFL,1287.0,24.55,35.0,M-SC,3.07,164333.0,-97885.0,167718.0,-3.13,-37.33,102.06,26.63,239.0,-0.58,1.13,2.17,XY24,NTT,MISC
24,EASEMYTRIP,26.4,-21.95,32.0,M-SC,20.27,75234.0,-101581.0,182375.0,-3.02,-57.45,242.41,45.70,205.0,-0.56,0.52,0.00,XY24,NTT,TRAVEL
28,GREENPANEL,537.0,192.64,35.0,M-SC,5.39,130540.0,-52538.0,131519.0,-1.65,-28.70,100.75,43.14,240.0,-0.40,0.90,22.79,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-41.96,38.0,M-SC,8.87,113675.0,-1123.0,95169.0,-0.76,-0.98,83.72,81.92,236.0,-0.01,0.78,34.57,XR,NTT,DURABLES
25,FINCABLES,1641.55,-18.12,50.0,M-SC,10.20,148319.0,675.0,158657.0,1.29,0.46,106.97,107.91,216.0,0.00,1.02,3.47,OX40N,ATH,CABLES
70,STARHEALTH,761.00,37.49,70.0,H-SC,15.51,283016.0,1893.0,130980.0,-0.42,0.67,46.28,47.26,171.0,0.01,1.94,51.63,XY24,NTT,INSURANCE
86,WIPRO,420.00,-14.25,52.0,M-LC,5.58,151311.0,366.0,109095.0,-0.13,0.24,72.10,72.51,99.0,0.00,1.04,6.28,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-18.12,50.0,M-SC,10.20,148319.0,675.0,158657.0,1.29,0.46,106.97,107.91,216.0,0.00,1.02,3.47,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,150.00,85.0,M-SC,0.64,163841.0,-10718.0,10748.0,1.73,-6.14,6.56,0.02,234.0,-1.00,1.12,42.51,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,101.92,52.0,M-SC,3.43,87030.0,-13747.0,13838.0,0.12,-13.64,15.90,0.09,253.0,-0.99,0.60,14.34,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-65.98,49.0,H-SC,2.36,222921.0,-46746.0,83083.0,-0.45,-17.33,37.27,13.47,158.0,-0.56,1.53,13.35,XY24,NTT,PAINTS
68,SIS,528.00,2018.13,46.0,H-SC,2.37,85476.0,-25556.0,49166.0,-0.30,-23.02,57.52,21.26,166.0,-0.52,0.59,15.49,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,STARHEALTH,761.00,37.49,70.0,H-SC,15.51,283016.0,1893.0,130980.0,-0.42,0.67,46.28,47.26,171.0,0.01,1.94,51.63,XY24,NTT,INSURANCE
63,SAMMAANCAP,326.00,22.90,61.0,M-SC,37.08,165294.0,16074.0,128103.0,4.27,10.77,77.50,96.62,245.0,0.13,1.13,78.88,XY25,NTT,FINANCE
86,WIPRO,420.00,-14.25,52.0,M-LC,5.58,151311.0,366.0,109095.0,-0.13,0.24,72.10,72.51,99.0,0.00,1.04,6.28,XR,NTT,IT
25,FINCABLES,1641.55,-18.12,50.0,M-SC,10.20,148319.0,675.0,158657.0,1.29,0.46,106.97,107.91,216.0,0.00,1.02,3.47,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.44,131202.0,7866.0,123763.0,0.98,6.38,94.33,106.72,139.0,0.06,0.90,31.25,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.44,131202.0,7866.0,123763.0,0.98,6.38,94.33,106.72,139.0,0.06,0.90,31.25,AR,ATH,MISC
25,FINCABLES,1641.55,-18.12,50.0,M-SC,10.20,148319.0,675.0,158657.0,1.29,0.46,106.97,107.91,216.0,0.00,1.02,3.47,OX40N,ATH,CABLES
86,WIPRO,420.00,-14.25,52.0,M-LC,5.58,151311.0,366.0,109095.0,-0.13,0.24,72.10,72.51,99.0,0.00,1.04,6.28,XR,NTT,IT
85,WHIRLPOOL,2270.00,-41.96,38.0,M-SC,8.87,113675.0,-1123.0,95169.0,-0.76,-0.98,83.72,81.92,236.0,-0.01,0.78,34.57,XR,NTT,DURABLES
87,ZYDUSLIFE,1286.17,-15.41,34.0,H-MC,5.59,206580.0,-2994.0,77674.0,-1.03,-1.43,37.60,35.63,119.0,-0.04,1.42,15.01,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,10.47,24.0,X-SC,9.78,84212.0,-44458.0,89753.0,-1.22,-34.55,106.58,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
77,TMPV,1065.00,-20.63,25.0,X-LC,0.27,244118.0,-32292.0,453449.0,-4.73,-11.68,185.75,152.37,3.0,-0.07,1.67,0.00,XY24,NTT,AUTO
27,GLAXO,3466.20,-19.65,30.0,X-MC,6.40,199448.0,164.0,77845.0,0.55,0.08,39.03,39.15,60.0,0.00,1.37,29.11,X40,ATH,PHARMA
23,DMART,5391.45,-17.28,31.0,X-LC,5.29,201810.0,-8597.0,67768.0,-0.43,-4.09,33.58,28.12,19.0,-0.13,1.38,18.57,X40N,ATH,FMCG
46,JSWINFRA,342.00,-18.84,32.0,X-MC,5.67,189911.0,-10436.0,40945.0,0.16,-5.21,21.56,15.23,66.0,-0.25,1.30,25.36,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-20.63,25.0,X-LC,0.27,244118.0,-32292.0,453449.0,-4.73,-11.68,185.75,152.37,3.0,-0.07,1.67,0.00,XY24,NTT,AUTO
43,ITC,452.00,-40.74,47.0,X-LC,0.91,199072.0,-1066.0,21958.0,-0.26,-0.53,11.03,10.44,5.0,-0.05,1.37,5.00,X40,NTT,FMCG
34,HONAUT,58357.33,-25.23,35.0,X-SC,1.00,106350.0,-21588.0,68723.0,-0.23,-16.87,64.62,36.84,90.0,-0.31,0.73,8.88,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-20.00,67.0,X-MC,1.19,184766.0,8028.0,46709.0,0.18,4.54,25.28,30.97,75.0,0.17,1.27,17.67,X40,ATH,INSURANCE
67,SIEMENS,4671.50,-0.76,63.0,H-LC,1.54,161630.0,-24465.0,71942.0,4.81,-13.15,44.51,25.51,49.0,-0.34,1.11,19.11,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,10.47,24.0,X-SC,9.78,84212.0,-44458.0,89753.0,-1.22,-34.55,106.58,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
77,TMPV,1065.00,-20.63,25.0,X-LC,0.27,244118.0,-32292.0,453449.0,-4.73,-11.68,185.75,152.37,3.0,-0.07,1.67,0.00,XY24,NTT,AUTO
54,PAGEIND,51769.93,-28.95,36.0,X-MC,7.78,157880.0,-5860.0,49195.0,-0.74,-3.58,31.16,26.47,55.0,-0.12,1.08,0.03,X40,ATH,APPARELS
9,BAJAJHFL,181.50,-18.58,36.0,X-MC,6.62,176120.0,-24386.0,121540.0,-0.14,-12.16,69.01,48.45,64.0,-0.20,1.21,1.56,X40N,ATH,FINANCE
55,PGHH,17907.65,-31.71,35.0,X-MC,2.08,194310.0,-6510.0,74304.0,-0.63,-3.24,38.24,33.76,57.0,-0.09,1.33,1.79,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,986.00,-52.07,35.0,X-SC,37.19,47663.0,-17343.0,168269.0,2.74,-26.68,353.04,232.18,83.0,-0.10,0.33,2.74,XY24,NTT,MISC
60,RELAXO,1176.00,-40.95,41.0,X-SC,3.23,80445.0,-64715.0,140642.0,0.41,-44.58,174.83,52.31,91.0,-0.46,0.55,7.68,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,10.47,24.0,X-SC,9.78,84212.0,-44458.0,89753.0,-1.22,-34.55,106.58,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-15.14,48.0,X-MC,3.56,88680.0,-1998.0,16902.0,0.75,-2.20,19.06,16.44,63.0,-0.12,0.61,17.53,X40,ATH,PHARMA
34,HONAUT,58357.33,-25.23,35.0,X-SC,1.00,106350.0,-21588.0,68723.0,-0.23,-16.87,64.62,36.84,90.0,-0.31,0.73,8.88,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-26.54,59.0,X-LC,8.48,291607.0,-54349.0,121046.0,-0.12,-15.71,41.51,19.28,1.0,-0.45,2.00,7.80,X40,ATH,IT
41,INFY,2275.00,-19.96,54.0,X-LC,4.06,321119.0,8231.0,163450.0,0.32,2.63,50.90,54.87,2.0,0.05,2.20,11.13,X40,BTT,IT
77,TMPV,1065.00,-20.63,25.0,X-LC,0.27,244118.0,-32292.0,453449.0,-4.73,-11.68,185.75,152.37,3.0,-0.07,1.67,0.00,XY24,NTT,AUTO
83,VBL,671.64,-17.80,49.0,X-LC,5.98,294565.0,-21277.0,133939.0,0.50,-6.74,45.47,35.67,4.0,-0.16,2.02,6.18,X40N,ATH,FMCG
43,ITC,452.00,-40.74,47.0,X-LC,0.91,199072.0,-1066.0,21958.0,-0.26,-0.53,11.03,10.44,5.0,-0.05,1.37,5.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7716.67,67.0,L-SC,17.21,85749.0,-8061.0,84677.0,1.13,-8.59,98.75,81.67,271.0,-0.10,0.59,66.06,XR,NTT,CERAMICS
51,MASFIN,398.61,-17.34,55.0,H-SC,7.50,93765.0,-4215.0,25814.0,1.07,-4.30,27.53,22.05,164.0,-0.16,0.64,36.29,XR,ATH,FINANCE
38,INDIAMART,4810.62,-47.39,55.0,H-SC,4.44,131202.0,7866.0,123763.0,0.98,6.38,94.33,106.72,139.0,0.06,0.90,31.25,AR,ATH,MISC
71,SURYODAY,240.00,60.09,51.0,H-SC,13.53,147390.0,-31681.0,97410.0,5.07,-17.69,66.09,36.71,167.0,-0.33,1.01,46.24,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,72.45,61.0,H-SC,9.45,149714.0,-33061.0,144130.0,0.13,-18.09,96.27,60.77,159.0,-0.23,1.03,34.83,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,150.00,85.0,M-SC,0.64,163841.0,-10718.0,10748.0,1.73,-6.14,6.56,0.02,234.0,-1.00,1.12,42.51,OX40N,NTT,PAINTS
81,VAIBHAVGBL,521.00,72.45,61.0,H-SC,9.45,149714.0,-33061.0,144130.0,0.13,-18.09,96.27,60.77,159.0,-0.23,1.03,34.83,XR,NTT,JEWELLERY
13,BSOFT,831.70,-9.16,64.0,H-SC,8.58,99667.0,-40986.0,112414.0,0.55,-29.14,112.79,50.79,151.0,-0.36,0.68,16.04,XR,ATH,IT
5,ASIANPAINT,3465.66,-0.07,83.0,X-LC,9.50,254135.0,2367.0,50852.0,-0.64,0.94,20.01,21.13,31.0,0.05,1.74,36.70,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-701.24,71.0,L-MC,6.28,48681.0,-28526.0,53919.0,0.68,-36.95,110.76,32.89,259.0,-0.53,0.33,34.06,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.57
1,25,44.26
2,50,75.08


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.82
MC    28.57
LC    24.59
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.05
X40      20.48
X40N     11.53
XY25     10.08
XR        9.57
AR        9.02
OX40N     7.62
X200      1.81
SR        1.01
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.31
X-MC    22.21
X-LC    19.65
M-SC    13.31
X-SC     7.78
H-MC     4.67
H-LC     2.92
L-SC     1.42
M-MC     1.36
M-LC     1.04
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.61,-3.89,37.48
IT,13.33,-14.59,73.80
FINANCE,10.46,-11.37,63.74
MISC,7.48,-19.31,79.49
PAINTS,6.00,-5.52,21.25
ELECTRICAL,5.63,-10.09,49.11
INSURANCE,4.51,3.89,29.98
PHARMA,3.40,-0.98,34.85
AUTO,3.35,-17.47,130.09


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3356492.0,21
XR,1259480.0,13
AR,1256785.0,10
X40,983360.0,14
X40N,688486.0,9
OX40N,681943.0,10
XY25,462603.0,7
SR,273688.0,2
MH,66466.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3461384.0,25
M-SC,1411609.0,16
X-LC,1201181.0,11
X-MC,1154430.0,16
X-SC,830696.0,8
H-MC,397749.0,3
L-SC,254890.0,3
H-LC,132842.0,2
M-LC,109095.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1165693.0      6
           AR         843895.0      5
           XR         784961.0      7
M-SC       XY24       747186.0      6
X-LC       XY24       517953.0      2
X-MC       X40        408125.0      7
X-LC       X40        406864.0      5
X-SC       XY24       400317.0      3
X-MC       XY24       344099.0      3
H-SC       OX40N      326681.0      4
           SR         273688.0      2
M-SC       OX40N      268417.0      5
X-SC       X40N       262008.0      3
X-MC       X40N       224771.0      4
H-MC       AR         216505.0      2
X-LC       X40N       201707.0      2
H-MC       XY24       181244.0      1
X-MC       XY25       177435.0      2
X-SC       X40        168371.0      2
L-SC       XR         168045.0      2
M-SC       XR         143460.0      2
           XY25       128103.0      1
           AR         124443.0      2
M-LC       XR         109095.0      1
L-SC       OX40N       86845.0      1
X-LC       XY25        74657.0      2
H-LC       AR          71942.0      1
H-SC       MH          66466.0      1
H-LC       X200        60900.0      1
L-MC       XR          53919.0      1
M-MC       XY25        46532.0      1
L-LC       XY25        35876.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
